In [1]:
from pypbc import *
import hashlib
import time

In [2]:
stored_params = """type a
q 8780710799663312522437781984754049815806883199414208211028653399266475630880222957078625179422662221423155858769582317459277713367317481324925129998224791
h 12016012264891146079388821366740534204802954401251311822919615131047207289359704531102844802183906537786776
r 730750818665451621361119245571504901405976559617
exp2 159
exp1 107
sign1 1
sign0 1
"""

params = Parameters(param_string=stored_params)
pairing = Pairing(params)

In [3]:
def setup():
    g = Element.random(pairing, G1)
    
    return g

g = setup()
g

0360C557CC938BC4EC0C2999DC39D5DC8DAF5F8581E883B7ECA9EF9D30EB15513C73DDEE9D22F73B81648EA69BD60ECB3A8BF3CAAE7C970CEE8D6D4EE03FDBA8C6

In [4]:
def keygen(g):
    x = Element.random(pairing, Zr)
    y = g**x
    
    return x, y

x, y = keygen(g)
y

029D2D0773063051879C2616A5FA6B76F9F51857173A6BDC91DCC604408D11109311DEC094D82800040A9DEE4F8206D9AB15BFEC3A28F61472AD867B8A2788C2C9

In [5]:
def get_hash(c1, c2, c3):
        h_in = str(c1) + str(c2) + str(c3)
        sha512 = hashlib.sha512()
        sha512.update(h_in.encode('utf8'))
        hv = sha512.hexdigest()
#         ev = Element.from_hash(pairing, G1, hv)
        
        return hv

In [ ]:
def xor(a, b):
    aa = int(str(a), 16)
    bb = int(str(b), 16)
    
    c = aa ^ bb
    cc = hex(c)
    c_e = Element(pairing, G1, value=str(cc))
    
    return c_e

p = Element.random(pairing, G1)
q = Element.random(pairing, G1)
xor(p, q)

In [2]:
def enc(m, g, y):
    r = Element.random(pairing, Zr)
    u = g**r
    v = m**r
    
    h = get_hash(u, v, y**r)
    mr_s = str(m) + str(r)
    w = ''
    for i in range(len(h)):
        w += chr(ord(h[i]) ^ ord(mr_s[i])) 
    
    return [str(u), str(v), w]
    
c1 = []
c2 = []
m = []
x1, y1 = keygen(g)
x2, y2 = keygen(g)

num = 9
for i in range(num):
    m.append(Element.random(pairing, G1))
    
start = time.time()
for i in range(num):
    c1.append(enc(m[i], g, y1))
enc_time = time.time() - start

for i in range(num):
    c2.append(enc(m[i], g, y2))
print(enc_time)
c1[0]

NameError: name 'keygen' is not defined

In [7]:
def dec(sk, c):
    u = Element(pairing, G1, value=c[0])
    v = Element(pairing, G1, value=c[1])
    h = get_hash(u, v, u**sk)
    mr_s = ''
    
    for i in range(len(h)):
        mr_s += chr(ord(h[i]) ^ ord(c[2][i]))
    
    m = Element(pairing, G1, value=mr_s)
    
    return m

c = enc(m[0], g, y)
st = time.time()
mr_s = dec(x, c)
dec_time = time.time() - st
print('dec time: ', dec_time)
print('c: ', c)
print('mr_s: ', mr_s)
print('m: ', m[0])

dec time:  0.002215862274169922
c:  ['0245E10FA7C9220C7EF3AC4B5F9BBA1ED51E23BD45F7618B9E2631EE90436F6F87DE531EF2E4A70786D460AE3A4A43B643A2FC1BF5B6CA64B229A8E531255A4697', '0309299BF6E0F994EDC27298EF4D4A5F2B7203BCC2E31CA0F79093056DBB95ABE7A46447220D984CB785E1BF35A26810C9B55A67D8B50941D985E371D52E1BA0A4', '\x06RV\t]$%\x03\x0c!\x02SQS \x0b~sV\x05X\x0ez\\s\x0b\x02U\x07\x01\x08Z"t%\x01R\x03x\x08qr\x06\x03\x00\x04[vV\n\x03\x07\x01"$tR\x07\x07\x01x\x00\x0f\nU\x07uwq\'$w\x07\x05\x01p!R\'\x02SY\x03%P\x06T%r\x04sQ&w#p\x0f"\x01\x01\x0c\']W\x0e\x04s\x02 qS\x06\x00vT\x05\x0c\x03Wt v\x03sS&\x04\x00']
mr_s:  03398AD58C3700A8FE5398C9B3774488AAA714A8BD32238E22716DFB3313A68801ADDABA308EE7C6186C425FA4F3DCEC8F305B8286C0EA761E60931BCD4A6E3901
m:  03398AD58C3700A8FE5398C9B3774488AAA714A8BD32238E22716DFB3313A68801ADDABA308EE7C6186C425FA4F3DCEC8F305B8286C0EA761E60931BCD4A6E3913


In [8]:
m = Element.random(pairing, G1)
r = Element.random(pairing, Zr)
u = g**r
v = m**r
    
h = get_hash(u, v, y**r)
mr_s = str(m) + str(r)
print('Length of h: ', len(h))
print('Length of m: ', len(str(m)))
print('Length of r: ', len(str(r)))
c = ''
for i in range(len(h)):
#     print(ord(h[i]) ^ ord(mr_s[i]))
    c += chr(ord(h[i]) ^ ord(mr_s[i])) 
len(c)
c

Length of h:  128
Length of m:  130
Length of r:  42


'Q\x02#W#q#vrv ws\x05RQ\x0b^\x02$p\x0eR\x05\x04ts\n\x04\x03zu}VU[\x00\x04#\'#P\x01vv\x02p\x06sR ^P\x0f\x05p\x03\x02\x01\x02\'\x0e\x04\rS^q\x0fp\x07VP$\x03_}\x04pvW\x06R\x04 YsU\x04"&$\x02\x00\t\x0f\x07\x01\x01WqqV\t\x08\x05\\\x0bW\\\x0fws\nU YQT rtQV\x03W\x03\x03\x07'

In [21]:
def test(c1, c2):
    u1 = Element(pairing, G1, value=c1[0])
    u2 = Element(pairing, G1, value=c2[0])
    v1 = Element(pairing, G1, value=c1[1])
    v2 = Element(pairing, G1, value=c2[1])
    
    start = time.time()
    if pairing.apply(u1, v2) == pairing.apply(u2, v1):
        ret = 1
    else:
        ret = 0

    end = time.time() - start

    return ret, end

d = []
test_time = 0
for i in range(len(c1)):
    t_ret, t_t = test(c1[i], c2[i])
    d.append(t_ret)
    test_time += t_t
    
print(test_time)
print(d)

0.05260872840881348
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [47]:
p = Element.random(pairing, G1)
q = Element.random(pairing, G1)
print(p)
print(q)

ps = int(str(p), 16)
qs = int(str(q), 16)
print(ps)
print(qs)
rv = ps ^ qs
rr = hex(rv)
print('rr: ', rr)
print(str(rr))
r = Element(pairing, G1, value=str(rr))
len(str(rv))

035B689AFC82A323B5412651E4770456054618DA2BF4E953FD7BBDAF699A99BBF3F83BD7EAC3944F8E206F09C1DB7AFF7F7805916263AD2648F893E891527AE300
022E4EE4470741A957D87CADA505D924968C6AF527C11683F04ABDFCF5514D42F0F99D038E16C1376A52D08113B22CA27612161012CEBBE7A14DBD69E7162DB024
45010881413964415059578805915666289935797225825825471317768124941855391208174415618958546425093577803921072997329240631303603362476530092770582107558437632
29240971558462268261758277812289481321448733630174612150849692598250419957613678769051350949466166154947952764132727274447557197831357226324068263816704036
rr:  0x175267ebb85e28ae2995afc4172dd7293ca722f0c35ffd00d3100539ccbd4f90301a6d464d55578e472bf88d269565d096a138170ad16c1e9b52e817644575324
0x175267ebb85e28ae2995afc4172dd7293ca722f0c35ffd00d3100539ccbd4f90301a6d464d55578e472bf88d269565d096a138170ad16c1e9b52e817644575324


155

In [30]:
a = Element.random(pairing, G1)
b = Element.random(pairing, G1)

s = time.time()
c = pairing.apply(a, b)
p_time = time.time() - s
print(p_time)

r = Element.random(pairing, Zr)
st = time.time()
d = a**r
z_time = time.time() - st
print(z_time)

0.003080129623413086
0.003310680389404297
